In [27]:
import pandas as pd
from capping import capping
from coinmarketcap import Market
from bittrex import Bittrex, API_V2_0, API_V1_1

# get coinmarketcap top 50
coinmarketcap = Market()
data = coinmarketcap.ticker(limit=50)

# create blacklist
blacklist = pd.Series(['BCC','USDT'])

"""
## bittrex

# get all bittrex markets
my_bittrex = Bittrex(None, None, api_version=API_V1_1)  # or defaulting to v1.1 as Bittrex(None, None)
bittrex_markets = pd.DataFrame(my_bittrex.get_markets()['result'])

# get coins traded in BTC in bittrex
bittrex_coins = bittrex_markets.loc[bittrex_markets['BaseCurrency'] == 'BTC']['MarketCurrency']
bittrex_coins = bittrex_coins.append(pd.Series('BTC'))

# create dataframe from coinmarketcap data
df = pd.DataFrame(data)

# remove blacklist coins and coins not in bittrex from dataframe
df = df.loc[df['symbol'].isin(bittrex_coins) & ~df['symbol'].isin(blacklist), :]
"""

## binance
from binance.client import Client
my_binance = Client(None, None)

# binance markets
binance_markets = my_binance.get_products()

binance_coins = pd.Series(list(set([p['baseAsset'] for p in binance_markets['data']])))
binance_coins = binance_coins.replace('IOTA', 'MIOTA')

# create dataframe from coinmarketcap data
df = pd.DataFrame(data)

# remove blacklist coins and coins not in bittrex from dataframe
df = df.loc[df['symbol'].isin(binance_coins) & ~df['symbol'].isin(blacklist), :]


# get top 20
df = df.head(15)

# compute market weights
df['market_cap_usd'] = df['market_cap_usd'].astype(float)
df['weight'] = df['market_cap_usd']/df['market_cap_usd'].sum()

# compute capped weights
capped = capping(df, 0.1, weight_column='weight')

weights = capped.loc[:, ['name', 'symbol', 'weight', 'price_btc']]


In [28]:
#investment_eur = 500
coinmarketcap = Market()
conversion_eur_btc = float(coinmarketcap.ticker('Bitcoin', convert='EUR')[0]['price_eur'])

#investment_btc = investment_eur/conversion_eur_btc
investment_btc = 0.01

In [29]:
weights['buy_btc'] = weights.weight*investment_btc
weights['buy_eur'] = weights.buy_btc*conversion_eur_btc
weights['buy_crypto'] = weights.buy_btc/weights.price_btc.astype(float)
#weights.set_index('symbol', inplace=True)
"""current_value = pd.Series({'BTC': 0, 
                           'ETH': 0.091908, 
                           'XRP': 76.923, 
                           'LTC': 0.21978, 
                           'MIOTA': 14.985, 
                           'DASH': 0.052947, 
                           'ADA': 177.822, 
                           'BTG': 0.10989, 
                           'XMR': 0.105894,
                           'EOS': 2.997,
                           'XLM': 121.822,
                           'NEO': 0.43956,
                           'ETC': 0.66933})"""
#weights['current_value'] = current_value
weights

,name,symbol,weight,price_btc,buy_btc,buy_eur,buy_crypto
0,Bitcoin,BTC,0.100000,1.0,0.001000,14.143913,0.001000
1,Ethereum,ETH,0.100000,0.0518869,0.001000,14.143913,0.019273
3,Ripple,XRP,0.100677,0.00006367,0.001007,14.239674,15.812321
4,Litecoin,LTC,0.100000,0.0196639,0.001000,14.143913,0.050855
5,IOTA,MIOTA,0.100677,0.00031292,0.001007,14.239674,3.217341
6,Cardano,ADA,0.093249,0.00002936,0.000932,13.189111,31.760686
7,Dash,DASH,0.086124,0.0905533,0.000861,12.181264,0.009511
9,Bitcoin Gold,BTG,0.055368,0.0270337,0.000554,7.831263,0.020481
10,Monero,XMR,0.052608,0.0277052,0.000526,7.440780,0.018988
11,EOS,EOS,0.045754,0.0006756,0.000458,6.471447,0.677239
